In [1]:
import pandas as pd
import numpy as np
import math
from jellyfish import jaro_winkler
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from py_common_subseq import find_common_subsequences
import numbers
import time
from collections import Counter 
from fuzzywuzzy import fuzz
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
#NOTES
#removing duplicate token-ID keys inplace caused some strange ID issue, so I am not removing duplicates anymore.  could be improved.
#unique_token_freq_max should really be derived dynamically by the number of tokens, some math
#jaro score threshold is too low.  Try correlation matrix to identify the best score to use, not sure it is jaro
#need to add code to remove duplicate ID pairs: A-B and B-A are the same

In [15]:
import io
import shutil

with io.open('training_data.txt', encoding='utf-8', errors='ignore') as source:
    with io.open('training_data_utf.txt', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

In [20]:
right_df = pd.read_table('mint orgs_utf.txt',keep_default_na=False,error_bad_lines=False)
right_df.head()

Skipping line 943: expected 10 fields, saw 11
Skipping line 1897: expected 10 fields, saw 11
Skipping line 2024: expected 10 fields, saw 11
Skipping line 2379: expected 10 fields, saw 11
Skipping line 9308: expected 10 fields, saw 11
Skipping line 12943: expected 10 fields, saw 11
Skipping line 20599: expected 10 fields, saw 11
Skipping line 22142: expected 10 fields, saw 11
Skipping line 22434: expected 10 fields, saw 11
Skipping line 23297: expected 10 fields, saw 11
Skipping line 24235: expected 10 fields, saw 11
Skipping line 24955: expected 10 fields, saw 11
Skipping line 25932: expected 10 fields, saw 11
Skipping line 26535: expected 10 fields, saw 11
Skipping line 26581: expected 10 fields, saw 11
Skipping line 29338: expected 10 fields, saw 11
Skipping line 31345: expected 10 fields, saw 11
Skipping line 32432: expected 10 fields, saw 11
Skipping line 35982: expected 10 fields, saw 11
Skipping line 37408: expected 10 fields, saw 11
Skipping line 40387: expected 10 fields, saw 1

,rec_id,cbi_orgid,NAME,STREET1,STREET2,CITY,STPROV,ZIP,PHONE,URL
0,1,NULL,1ST ARMORED DIVISION ASSOCIATION,1815 LAKEWOOD DR,,ELIZABETHTOWN,KY,42702,502-737-0901,unknown
1,2,NULL,1ST CANADIAN PARACHUTE BATTALION ASSOCIATION,360 BLOOR ST E,APT 610,TORONTO,ON,M4W 3M3,416-927-0118,NULL
2,3,96749,1st Cavalry Division Association,302 North Main Street,,Copperas Cove,TX,76522,(254) 547-6537,http://www.1cda.org
3,4,83774,1st Fighter Group Association,11512 Henegan Pl.,NULL,Spotsylvania,VA,22551,308-632-3946,1stfighterassociation.weebly.com/
4,5,85835,1st Marine Division Association,1132 E. St.,Camp Pendleton,North,CA,92055,760-763-3268,1stmarinedivisionassociation.org


In [2]:
overall_time = time.time()

#define column names
l_id = 'organization_id'
l_name = 'org_name'
l_address1 = 'address1'
l_city = 'city'
l_state = 'state'
l_zip = 'postal_code'
l_phone = 'phone'
l_web = 'website'
l_acronym = 'acronym'
l_alt_name = 'alt_name'

r_id = 'rec_id'
r_name = 'NAME'
r_address1 = 'STREET1'
r_city = 'CITY'
r_state = 'STPROV'
r_zip = 'ZIP'
r_phone = 'PHONE'
r_web = 'URL'
#r_acronym = 'CBI_Acronym'

#set parameters
token_match_min = 3 # minimum number of matched tokens to be considered a match
token_limiter = .999 # percent of non-single tokens to tokenize, where rare tokens are at the bottom and common at the top
unique_token_freq_max = 5 #threshold <= to a token is considered "unique" and links to these tokens are counted double
name_weight = .75 #note that this is really .75 * 4 because there are 4 org name simularity metrics
name_uniqueness_weight = 1.5
state_weight = 1
zip_weight = 1
phone_weight = 2
domain_weight = 2
name_score_min = 3
composite_score_min = 4 #minimum composite match score to be considered a match

start_time = time.time()
print "LOADING INITIAL DATAFRAMES..."

left_df = pd.read_csv('all cupola orgs w phone_db_utf.csv',keep_default_na=False)
right_df = pd.read_table('mint orgs_utf.txt',keep_default_na=False,error_bad_lines=False)

left_df.rename(columns={l_id:'id',l_name:'l_org_name',l_address1:'l_address1',l_city:'l_city',l_state:'l_state',l_zip:'l_postal_code',l_web:'l_web',l_phone:'l_phone',l_acronym:'l_acronym',l_alt_name:'l_alt_name'}, inplace=True)
right_df.rename(columns={r_id:'id',r_name:'r_org_name',r_address1:'r_address1',r_city:'r_city',r_state:'r_state',r_zip:'r_postal_code',r_web:'r_web',r_phone:'r_phone'}, inplace=True)

print("Dataframes loaded --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "PRE-PROCESSING..."
#normalize state codes
state_lkup = pd.read_csv('state_lkup.csv',keep_default_na=False)

from collections import defaultdict
state_dict = defaultdict(list)
for state, acronym in zip(state_lkup.state.values,state_lkup.acronym.values):
    state_dict[state].append(acronym)

left_df.l_state = left_df.l_state.str.lower()
left_df.l_state = left_df.l_state.replace(state_dict)
right_df.r_state = right_df.r_state.str.lower()
right_df.r_state = right_df.r_state.replace(state_dict)

#clean up non numeric characters in phones
l_clean_phones = []
for phone in left_df.l_phone:
    l_clean_phones.append(re.sub('[^0-9]','', phone))
    
left_df['l_clean_phone'] = l_clean_phones

r_clean_phones = []
for phone in right_df.r_phone:
    r_clean_phones.append(re.sub('[^0-9]','', phone))

right_df['r_clean_phone'] = r_clean_phones

#isolate domains from web URLs
l_domains = []
for web in left_df.l_web:
    l_domains.append(web.split('//')[-1].split('/')[0].strip('www.'))
left_df['l_domain'] = l_domains
    
r_domains = []
for web in right_df.r_web:
    r_domains.append(web.split('//')[-1].split('/')[0].strip('www.'))
right_df['r_domain'] = r_domains

print("states, phones, domains normalized --- %s seconds ---" % (time.time() - start_time))
print ""


start_time = time.time()
print "TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS..."

left_unique_token_columns = [ #tokens here circumvent the token_limiter which is cutting off the most commonly occuring tokens
    'l_state', 
    'l_postal_code',
    'l_clean_phone'

]

left_delta_token_columns = [
    'l_acronym',
    'l_org_name',
    'l_alt_name',
    #'l_address1',
    #'l_address2',
    'l_city',
    'l_domain'
]

right_unique_token_columns = [
    'r_state', 
    'r_postal_code',
    'r_clean_phone'
]

right_delta_token_columns = [
    #'r_acronym',
    'r_org_name',
    #'r_alt_name',
    #'r_address1',
    #'r_address2',
    'r_city',
    'r_domain'
]

# lowercase the name and split on spaces, remove non-alphanumeric chars
def tokenize_name(name):
    if isinstance(name, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in name)
        return clean_name.lower().split()
    else:
        return name
    
unique_tokens = []    
for col in left_unique_token_columns:
    for word in left_df[col]:
        if isinstance(word, float) is False:
            unique_tokens.append(tokenize_name(str(word)))
            
for col in right_unique_token_columns:
    for word in right_df[col]:
        if isinstance(word, float) is False:
            unique_tokens.append(tokenize_name(str(word)))
            
unique_flat_list = [item for sublist in unique_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
u_cnt = Counter()
for token in unique_flat_list:
    u_cnt[token] += 1

u_cnt_dict = dict(u_cnt) #convert to dictionary

unique_tokens_df = pd.DataFrame(u_cnt_dict.items(), columns=['token', 'count'])
unique_tokens_df = unique_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#consider waiting to do the count flag thing later, instead use some type of "token type" code
unique_token_flag = []
for index, value in enumerate(unique_tokens_df['count']):
    if value == 1:
        unique_token_flag.append(0)  #for any tokens occuring only once, we exclude
    else:
        unique_token_flag.append(1)

unique_tokens_df['flag'] = unique_token_flag        

all_other_words = []
for col in left_delta_token_columns:
    for word in left_df[col]:
        if isinstance(word, float) is False:
            all_other_words.append(tokenize_name(str(word)))
            
for col in right_delta_token_columns:
    for word in right_df[col]:
        if isinstance(word, float) is False:
            all_other_words.append(tokenize_name(str(word)))
            
flat_list = [item for sublist in all_other_words for item in sublist] #flatten list so it can be counted

#instantiate counter and use to count word frequencies in flat list
cnt = Counter()
for token in flat_list:
    cnt[token] += 1

cnt_dict = dict(cnt) #convert to dictionary

main_tokens_df = pd.DataFrame(cnt_dict.items(), columns=['token', 'count'])
main_tokens_df = main_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#wait to do count until joined with unique tokens?
main_token_flag = []
for index, value in enumerate(main_tokens_df['count']):
    if value == 1:
        main_token_flag.append(0)  #for any tokens occuring only once, we exclude
    elif index < int(main_tokens_df.shape[0] * token_limiter): #important line, we are cutting the top x% of frequently occuring tokens
        main_token_flag.append(1)
    else:
        main_token_flag.append(0)  #for the most common tokens, we exclude

main_tokens_df['flag'] = main_token_flag

all_tokens = pd.concat([unique_tokens_df, main_tokens_df])

all_tokens.drop('count',axis=1,inplace=True)
all_tokens['flag'] = all_tokens.flag.astype(int) #converting flags to int
tokens_dct = all_tokens.to_dict('split') #converting tokens_df to dictionary
tokens_dct=dict(tokens_dct['data']) #honestly can't remember why this works, something to do with conversion to dictionary

#preparing token_ids which will be used for joining left and right dfs
all_tokens.sort_values(by='flag',ascending=False,inplace=True)
all_tokens.sort_values(by='token',inplace=True)
all_tokens.drop_duplicates(subset='token',keep='first',inplace=True)
token_ids = all_tokens.index.get_level_values(0)
all_tokens['token_id'] = token_ids

all_tokens.drop('flag',axis=1,inplace=True)
all_tokens['token_id'] = all_tokens.token_id.astype(int)
token_id_dct = all_tokens.to_dict('split')
tokens_id_dct=dict(token_id_dct['data'])

vocabulary = np.array([w for w, c in tokens_dct.items() if c ==1]) #this works even without the ==1 and I don't know why
cv = CountVectorizer( vocabulary=vocabulary)

#now we are ready to tokenize left and right dataframes
all_left_cols = left_unique_token_columns + left_delta_token_columns

left_frame_list = []
for colname in all_left_cols:
    tokenmapping = cv.fit_transform(left_df[colname])
    df_row, token_id = tokenmapping.nonzero()

    left_frame_list.append(pd.DataFrame(np.vstack([vocabulary[token_id], left_df['id'].values[df_row]]).T, columns = ['token', 'id_l']))

left_keyed = pd.concat(left_frame_list)
left_keyed.drop_duplicates()#inplace=True)
#removing duplicates inplace was giving me a very strange issue where a small percentage of token_ids would be excluded from the left_keyed index

#append token_id to token as this will be more efficient to join with
left_token_ids = []
for token in left_keyed.token:
    left_token_ids.append(tokens_id_dct[token])

left_keyed['token_id'] = left_token_ids
left_keyed.sort_values(by='token_id',inplace=True)
left_keyed.set_index('token_id',inplace=True)
left_keyed.drop('token',axis=1,inplace=True)

all_right_cols = right_unique_token_columns + right_delta_token_columns

right_frame_list = []
for colname in all_right_cols:
    tokenmapping = cv.fit_transform(right_df[colname])
    df_row, token_id = tokenmapping.nonzero()

    right_frame_list.append(pd.DataFrame(np.vstack([vocabulary[token_id], right_df['id'].values[df_row]]).T, columns = ['token', 'id_r']))

right_keyed = pd.concat(right_frame_list)
right_keyed.drop_duplicates()#inplace=True)

#append token_id to token as this will be more efficient to join with
right_token_ids = []
for token in right_keyed.token:
    right_token_ids.append(tokens_id_dct[token])

right_keyed['token_id'] = right_token_ids
right_keyed.sort_values(by='token_id',inplace=True)
right_keyed.set_index('token_id',inplace=True)
right_keyed.drop('token',axis=1,inplace=True)

aggregations = {
    'id_l': 'count'
}

joined = left_keyed.join(right_keyed, how='inner',lsuffix='_l',rsuffix='_r')
#double-counting unique token matches
bonus_point_tokens = []
for token in main_tokens_df[(main_tokens_df['count'] > 1) & (main_tokens_df['count']<=unique_token_freq_max)].token:
    bonus_point_tokens.append(tokens_id_dct[token])

intersection_bonus_tokens = set(bonus_point_tokens).intersection(set(list(joined.index)))    
    
bonus_token_joins = []
for token_id in intersection_bonus_tokens:
    bonus_token_joins.append(joined.loc[token_id])
    
bonus_joins = pd.concat(bonus_token_joins)
bonus_joins_cols = bonus_joins[['id_l','id_r']].copy()
bonus_joins_cols.dropna(inplace=True)
joined_bonus = pd.concat([joined,bonus_joins_cols])

keys_grouped = joined_bonus.groupby(by=['id_l', 'id_r']).agg(aggregations)
keys_grouped.rename(columns={'id_l':'id_l count'}, inplace=True)
matched_records = keys_grouped[keys_grouped['id_l count'] >= token_match_min]
matched_records.reset_index(inplace=True)

print("match candidates identified --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
left_df.rename(columns={'id':'id_l'},inplace=True)
right_df.rename(columns={'id':'id_r'},inplace=True)

left_match_data = left_df[['id_l','l_org_name','l_city','l_state','l_postal_code','l_domain','l_clean_phone']].copy()
right_match_data = right_df[['id_r','r_org_name','r_city','r_state','r_postal_code','r_domain','r_clean_phone']].copy()

#making sure keys are str, results in blank df otherwise
left_match_data.id_l = left_match_data.id_l.astype('str')
right_match_data.id_r = right_match_data.id_r.astype('str')
matched_records.id_l = matched_records.id_l.astype('str')
matched_records.id_r = matched_records.id_r.astype('str')

#merging matched_records df with original record data for ease of review
l_conc = pd.merge(matched_records, left_match_data, on='id_l')
full_conc = pd.merge(l_conc, right_match_data, on='id_r')

print("original data concatenated with matches --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING ORG NAME SIMULARITY..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on edit distance of org names
def jaro_simularity(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return jaro_winkler(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '')
def fuzz_partial(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.partial_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_sort(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_sort_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_set(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_set_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)

full_conc['l_org_name'] = full_conc['l_org_name'].astype('str')
full_conc['r_org_name'] = full_conc['r_org_name'].astype('str')

jaro_time = time.time()
full_conc['jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_org_name, x.r_org_name), axis=1)
print("jaro scores done --- %s seconds ---" % (time.time() - jaro_time))

jaro_reduced = full_conc[full_conc.jaro_score > .25]

partial_time = time.time()
jaro_reduced['fuzz_partial_score'] = jaro_reduced.apply(lambda x: fuzz_partial(x.l_org_name, x.r_org_name), axis=1)
print("fuzz partial scores done --- %s seconds ---" % (time.time() - partial_time))
sort_time = time.time()
jaro_reduced['fuzz_sort_score'] = jaro_reduced.apply(lambda x: fuzz_sort(x.l_org_name, x.r_org_name), axis=1)
print("fuzz sort scores done --- %s seconds ---" % (time.time() - sort_time))
set_time = time.time()
jaro_reduced['fuzz_set_score'] = jaro_reduced.apply(lambda x: fuzz_set(x.l_org_name, x.r_org_name), axis=1)
print("fuzz set scores done --- %s seconds ---" % (time.time() - set_time))
print ""

print("name simularity scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING NAME SEQUENCE UNIQUENESS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

org_tokens = []    
for word in left_df['l_org_name']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))

for word in right_df['r_org_name']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))
            
org_flat_list = [item for sublist in org_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
org_cnt = Counter()
for token in org_flat_list:
    org_cnt[token] += 1

org_cnt_dict = dict(org_cnt) #convert to dictionary

def sequence_uniqueness(seq):
    return sum(1/org_cnt_dict[str.lower(t)]**0.5 for t in seq)

def name_similarity(a, b):
    if isinstance(a,basestring) is True and isinstance(b,basestring) is True:
        if len(a) > 0 and len(b) > 0:
            a_tokens = set(tokenize_name(a))
            b_tokens = set(tokenize_name(b))
            a_uniq = sequence_uniqueness(a_tokens)
            b_uniq = sequence_uniqueness(b_tokens)

            return sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
    else:
        return 0

jaro_reduced['uniq'] = jaro_reduced.apply(lambda x: name_similarity(x.l_org_name, x.r_org_name), axis=1)

print("name uniqueness scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR STATE CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def sanitize_state(state):
    if isinstance(state,basestring) is True:
        return ''.join(c for c in (state or '') if c in 'abcdefghijklmnopqrstuvwxyz')
    else:
        return ''
    
def state_match(state_a, state_b):
    sanitized_state_a = str(sanitize_state(state_a))
    sanitized_state_b = str(sanitize_state(state_b))

    # if the value is too short, means it's fubar
    if len(sanitized_state_a) < 2 or len(sanitized_state_b) < 2:
        return 0
    if state_a == state_b:
        return 1
    else:
        return 0    

jaro_reduced['state_match'] = jaro_reduced.apply(lambda x: state_match(x.l_state, x.r_state), axis=1)

print("state codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR POSTAL CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching postal code

def sanitize_postal(postal):
    if isinstance(postal, basestring) is True:
        return ''.join(c for c in (postal or '') if c in '1234567890')
    if isinstance(postal, float) is False:
        return postal

def postal_simularity(postal_a, postal_b):
    sanitized_postal_a = str(sanitize_postal(postal_a))
    sanitized_postal_b = str(sanitize_postal(postal_b))

    # if the number is too short, means it's fubar
    if len(sanitized_postal_a) < 5 or len(sanitized_postal_b) < 5:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(sanitized_postal_a, sanitized_postal_b))) / 5 >= 1:
        return 1
    else:
        return 0
    
jaro_reduced['zip_match'] = jaro_reduced.apply(lambda x: postal_simularity(x.l_postal_code, x.r_postal_code), axis=1)
    
print("postal codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR WEB DOMAIN MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def domain_match(domain_a, domain_b):
    if isinstance(domain_a, basestring) is True and isinstance(domain_b, basestring) is True:
        if len(domain_a) > 0 and len(domain_b) > 0:
            if domain_a == domain_b:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0


jaro_reduced['domain_match'] = jaro_reduced.apply(lambda x: domain_match(x.l_domain, x.r_domain), axis=1)

print("web domains checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR PHONE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching phone
def phone_simularity(phone_a, phone_b):

    if len(phone_a) < 10 or len(phone_b) < 10:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(phone_a, phone_b))) / 10 >= 1:
        return 1
    else:
        return 0
    
jaro_reduced['phone_match'] = jaro_reduced.apply(lambda x: phone_simularity(x.l_clean_phone, x.r_clean_phone), axis=1)
    
print("phones checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "COMPOSITE SCORING, PREDICTING MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

jaro_reduced['overall_name_score'] = jaro_reduced.jaro_score * name_weight \
+ jaro_reduced.fuzz_partial_score * name_weight \
+ jaro_reduced.fuzz_sort_score * name_weight \
+ jaro_reduced.fuzz_set_score * name_weight \
+ jaro_reduced.uniq * name_uniqueness_weight

#calculate composite match score based on component scores and weights
jaro_reduced['composite_match_score'] = jaro_reduced.overall_name_score \
+ jaro_reduced.zip_match * zip_weight \
+ jaro_reduced.state_match * state_weight \
+ jaro_reduced.domain_match * domain_weight \
+ jaro_reduced.phone_match * phone_weight

#use labeled matches to train logistic regression to predict matches
training_data = pd.read_table('training_data_utf.txt')

feature_cols = ['jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                'domain_match',
                'phone_match']

# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(jaro_reduced[feature_cols])
y_pred_proba = log.predict_proba(jaro_reduced[feature_cols])[:,1]

jaro_reduced['match_pred'] = y_pred_class
jaro_reduced['pred_proba'] = y_pred_proba

#we take any matches meeting either name match threshold or composite match threshold as matches for review
#org_matches = jaro_reduced[(jaro_reduced.overall_name_score >= name_score_min) | (jaro_reduced.composite_match_score >= composite_score_min)]
org_matches = jaro_reduced[jaro_reduced.match_pred == 1]
failed_matches_for_review = jaro_reduced[(jaro_reduced.match_pred == 0) & (jaro_reduced.pred_proba > .4)]

print("final matches isolated --- %s seconds ---" % (time.time() - start_time))
print ""

print("TOTAL COMPUTE TIME --- %s seconds ---" % (time.time() - overall_time))

org_matches.sort_values(by='pred_proba', ascending=True)

LOADING INITIAL DATAFRAMES...


Skipping line 943: expected 10 fields, saw 11
Skipping line 1897: expected 10 fields, saw 11
Skipping line 2024: expected 10 fields, saw 11
Skipping line 2379: expected 10 fields, saw 11
Skipping line 9308: expected 10 fields, saw 11
Skipping line 12943: expected 10 fields, saw 11
Skipping line 20599: expected 10 fields, saw 11
Skipping line 22142: expected 10 fields, saw 11
Skipping line 22434: expected 10 fields, saw 11
Skipping line 23297: expected 10 fields, saw 11
Skipping line 24235: expected 10 fields, saw 11
Skipping line 24955: expected 10 fields, saw 11
Skipping line 25932: expected 10 fields, saw 11
Skipping line 26535: expected 10 fields, saw 11
Skipping line 26581: expected 10 fields, saw 11
Skipping line 29338: expected 10 fields, saw 11
Skipping line 31345: expected 10 fields, saw 11
Skipping line 32432: expected 10 fields, saw 11
Skipping line 35982: expected 10 fields, saw 11
Skipping line 37408: expected 10 fields, saw 11
Skipping line 40387: expected 10 fields, saw 1

Dataframes loaded --- 0.592000007629 seconds ---

PRE-PROCESSING...
states, phones, domains normalized --- 1.09000015259 seconds ---

TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS...
match candidates identified --- 359.305000067 seconds ---


C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


original data concatenated with matches --- 11.631000042 seconds ---

SCORING ORG NAME SIMULARITY...
jaro scores done --- 299.105999947 seconds ---


...

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


phones checked --- 896.59800005 seconds ---

COMPOSITE SCORING, PREDICTING MATCHES...


C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:502: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:505: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

final matches isolated --- 6.86899995804 seconds ---

TOTAL COMPUTE TIME --- 4698.579 seconds ---


,id_l,id_r,id_l count,l_org_name,l_city,l_state,l_postal_code,l_domain,l_clean_phone,r_org_name,...,fuzz_set_score,uniq,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,match_pred,pred_proba
5413535,77995,44780,3,American Traffic Safety Services Association- ...,Chandler,az,85224,atssa.com,6026970139,American Traffic Safety Services Association -...,...,0.93,0.702634,0,0,1,0,3.738426,5.738426,1,0.500041
4825933,95364,1175,9,World At Work,Scottsdale,az,85260,orldatwork.org,4809519191,WorldatWork,...,0.50,0.000000,1,1,1,1,1.999248,7.999248,1,0.500147
4826604,11200,45127,9,WorldatWork,Scottsdale,az,85260-3627,orldatwork.org,4809519191,World At Work,...,0.50,0.000000,1,1,1,1,1.999248,7.999248,1,0.500147
3824844,78825,50826,4,Society for Adolescent Medicine - Heartland Re...,Kansas City,mo,64111,adolescenthealth.org,8169603040,New England Regional Chapter - Society for Ado...,...,0.91,0.706255,0,0,1,0,3.433425,5.433425,1,0.500167
4114698,89617,49504,4,MCI Group,Plano,tx,75093,mci-group.com,9723497300,MCI USA,...,0.63,0.919506,0,0,1,0,3.068228,5.068228,1,0.500223
4232123,79215,38076,3,Mid-Atlantic Consortium for Human Services,Virginia Beach,va,23454,nationalhumanservices.org,5035952736,National Organization for Human Services,...,0.68,0.472156,0,0,1,1,2.704335,6.704335,1,0.500255
4471963,75739,45992,4,American Small Business Coalition,Columbia,md,21045,theasbc.org,,"American Small Business Alliance, Inc.",...,0.84,0.814883,1,1,0,0,3.712266,5.712266,1,0.500266
5324648,77074,29450,7,National Association of Postmasters of the Uni...,Alexandria,va,22305,ipostmasters.org,7152894227,National League of Postmasters of the United S...,...,0.92,0.804706,1,1,0,0,3.700305,5.700305,1,0.500297
5460974,79110,39379,3,Society of Critical Care Medicine - Michigan,Dearborn,mi,48124,sccm.org,8663093332,Society of Critical Care Medicine Carolina/Vir...,...,0.88,0.805250,0,0,1,0,3.657068,5.657068,1,0.500368
5141806,73163,35358,4,American Congress of Obstetricians and Gynecol...,Washington,dc,20024-9998,acog.org,2026385577,American College of Obstetricians and Gynecolo...,...,0.91,0.759375,1,1,0,0,3.769200,5.769200,1,0.500395


In [4]:
org_matches.to_csv('org_matches_w_preds.csv')

In [5]:
failed_matches_for_review.to_csv('failed_matches_for_review.csv')

In [49]:
jaro_reduced[jaro_reduced.domain_match ==1]

,id_l,id_r,id_l count,l_org_name,l_city,l_state,l_postal_code,l_domain,l_clean_phone,r_org_name,...,fuzz_set_score,uniq,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,domain_match2,domain_match3
73,91832,10224,12,Phi Delta Theta,Oxford,oh,45056-1801,phideltatheta.org,5135236345,Phi Delta Theta,...,1.00,1.000000,1,1,1.0,1,4.500000,10.500000,1.0,1
144,91836,10227,9,Phi Kappa Tau,Oxford,oh,45056,phikappatau.org,5135234193,Phi Kappa Tau,...,1.00,1.000000,1,1,1.0,1,4.500000,10.500000,1.0,1
154,20007,10292,3,Forum Health,Youngstown,oh,NULL,NULL,,PLANNING FORUM-THE INTERNATIONAL SOC/STRATEGIC...,...,0.59,0.304824,1,0,1.0,0,1.477432,4.477432,1.0,1
168,355,10292,4,Cleveland State University (OH),Cleveland,oh,44115,NULL,,PLANNING FORUM-THE INTERNATIONAL SOC/STRATEGIC...,...,0.37,0.000000,1,0,1.0,0,0.890758,3.890758,1.0,1
181,48336,10292,4,Bay Village Senior High School (OH),Bay Village,oh,,NULL,,PLANNING FORUM-THE INTERNATIONAL SOC/STRATEGIC...,...,0.29,0.000000,1,0,1.0,0,0.824318,3.824318,1.0,1
210,74349,10292,3,Calfee Strategic Solutions,Cincinnati,oh,45202,NULL,,PLANNING FORUM-THE INTERNATIONAL SOC/STRATEGIC...,...,0.51,0.097008,1,0,1.0,0,1.166179,4.166179,1.0,1
214,82358,10292,3,OH Strategic Communications,Phoenix,az,85012,NULL,,PLANNING FORUM-THE INTERNATIONAL SOC/STRATEGIC...,...,0.50,0.160558,0,0,1.0,0,1.325118,3.325118,1.0,1
448,16415,13943,20,Wilberforce University (OH),Wilberforce,oh,45384,ilberforce.edu,9373762911,Wilberforce University (OH),...,1.00,1.000000,1,1,1.0,1,4.500000,10.500000,1.0,1
2673,49533,14679,10,Academy of Legal Studies in Business,Oxford,oh,45056,alsb.org,,Academy of Legal Studies in Business,...,1.00,1.000000,1,1,1.0,0,4.500000,8.500000,1.0,1
4192,58213,15015,15,Ohio Federation of Teachers,Columbus,oh,43205-1487,oh.aft.org,6142583240,Ohio Federation of Teachers,...,1.00,1.000000,1,1,1.0,1,4.500000,10.500000,1.0,1


In [36]:
left_df[left_df.id_l ==1006]

,id_l,l_org_name,l_acronym,l_address1,address2,l_city,l_state,l_postal_code,l_alt_name,l_web,l_phone,l_clean_phone,l_domain
38528,1006,Michigan State University,,1100 New Jersey Ave. SE,Suite 735,Washington,dc,20003,NULL,www.msu.edu,517-355-1855,5173551855,msu.edu
87344,1006,Michigan State University,,"426 Auditorium Rd,",NULL,East Lansing,mi,48824-1046,NULL,www.msu.edu,517-355-1855,5173551855,msu.edu


In [2]:
training_data = pd.read_table('training_data_utf.txt')

In [17]:
training_data.head()

,Unnamed: 0,id_l,id_r,id_l count,l_org_name,l_city,l_state,l_postal_code,l_domain,l_clean_phone,...,fuzz_sort_score,fuzz_set_score,uniq,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,is_match
0,19778,1000,19406,7,Miami University of Ohio,Oxford,oh,45056,muohio.edu,5.135292e+09,...,1.0,1.0,1.0,1,1,1,1,4.5,10.5,1
1,4173438,1006,7492,11,Michigan State University,East Lansing,mi,48824-1046,msu.edu,5.173552e+09,...,1.0,1.0,1.0,1,1,1,1,4.5,10.5,1
2,3339786,100749,45684,14,Oxford House World Services,Silver Spring,md,20910,oxfordhouse.org,3.015873e+09,...,1.0,1.0,1.0,1,1,1,1,4.5,10.5,1
3,4451309,100752,46206,11,Oregon Association of Conservation Districts,Salem,or,97309,oacd.org,5.035669e+09,...,1.0,1.0,1.0,1,1,1,1,4.5,10.5,1
4,4454552,100753,46351,8,Pacific Landscape Management,Hillsboro,or,97124,pacscape.com,5.036484e+09,...,1.0,1.0,1.0,1,1,1,1,4.5,10.5,1


In [20]:
training_data.dtypes

Unnamed: 0                 int64
id_l                       int64
id_r                       int64
id_l count                 int64
l_org_name                object
l_city                    object
l_state                   object
l_postal_code             object
l_domain                  object
l_clean_phone            float64
r_org_name                object
r_city                    object
r_state                   object
r_postal_code             object
r_domain                  object
r_clean_phone            float64
jaro_score               float64
fuzz_partial_score       float64
fuzz_sort_score          float64
fuzz_set_score           float64
uniq                     float64
state_match                int64
zip_match                  int64
domain_match               int64
phone_match                int64
overall_name_score       float64
composite_match_score    float64
is_match                   int64
dtype: object

In [54]:
feature_cols = ['jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                #'domain_match',
                'phone_match']

In [33]:
# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# split the new DataFrame into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

# logistic regression
log = LogisticRegression()
log.fit(X_train, y_train)
y_pred_class = log.predict(X_test)

# calculate accuracy
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.recall_score(y_test, y_pred_class))
print(metrics.precision_score(y_test, y_pred_class))

0.9727272727272728
0.7681159420289855
0.8688524590163934


In [64]:
# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(jaro_reduced[feature_cols])
y_pred_proba = log.predict_proba(jaro_reduced[feature_cols])[:,1]

jaro_reduced['match_pred'] = y_pred_class
jaro_reduced['pred_proba'] = y_pred_proba

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [65]:
jaro_reduced.head()

,id_l,id_r,id_l count,l_org_name,l_city,l_state,l_postal_code,l_domain,l_clean_phone,r_org_name,...,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,domain_match2,domain_match3,match_pred,pred_proba
0,1000,10224,4,Miami University of Ohio,Oxford,oh,45056,muohio.edu,5135291809,Phi Delta Theta,...,1,1,NaN,0,1.155655,NaN,0.0,0,0,0.007744
1,1179,10224,4,Oberlin College (OH),Oberlin,oh,NULL,oberlin.edu,,Phi Delta Theta,...,1,0,NaN,0,0.952500,NaN,0.0,0,0,0.001842
2,14795,10224,4,"Cleveland, Ohio, City of",Cleveland,oh,44114,city.cleveland.oh.us,,Phi Delta Theta,...,1,0,NaN,0,1.110139,NaN,0.0,0,0,0.001938
3,15615,10224,4,"Montgomery County, Ohio/Montgomery County Soli...",Dayton,oh,NULL,co.montgomery.oh.us,9372254999,Phi Delta Theta,...,1,0,NaN,0,0.959341,NaN,0.0,0,0,0.001587
4,16415,10224,4,Wilberforce University (OH),Wilberforce,oh,45384,ilberforce.edu,9373762911,Phi Delta Theta,...,1,0,NaN,0,0.941389,NaN,0.0,0,0,0.001358


In [62]:
len(y_pred_proba)

5511718

In [76]:
org_matches.shape

(115492, 26)

In [78]:
jaro_reduced[jaro_reduced.match_pred == 1].shape

(52401, 30)

In [56]:
jaro_reduced['match_pred'] = y_pred_class
jaro_reduced['pred_proba'] = y_pred_proba

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
jaro_reduced[(jaro_reduced.match_pred == 0) & (jaro_reduced.pred_proba > .4)].shape

(8378, 30)

In [40]:
training_data.corr()

,Unnamed: 0,id_l,id_r,id_l count,l_clean_phone,r_clean_phone,jaro_score,fuzz_partial_score,fuzz_sort_score,fuzz_set_score,uniq,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,is_match
Unnamed: 0,1.000000,-0.212850,0.145518,0.079754,-0.106469,-0.002840,0.241467,0.196616,0.183734,0.241088,0.213345,-0.120406,-0.247624,0.009720,0.084502,0.234694,0.022994,0.076775
id_l,-0.212850,1.000000,-0.037245,0.112404,-0.033208,0.007791,-0.152362,-0.074333,0.052334,-0.098783,-0.137879,0.466644,0.232468,0.117771,0.053478,-0.103920,0.184262,0.106190
id_r,0.145518,-0.037245,1.000000,0.151777,-0.047452,-0.005460,0.394864,0.373749,0.244038,0.264388,0.274331,-0.073592,-0.191963,0.158681,0.054479,0.328630,0.131296,0.159218
id_l count,0.079754,0.112404,0.151777,1.000000,-0.181321,-0.011031,0.405832,0.434351,0.504531,0.457027,0.555493,0.037917,0.167583,0.421059,0.426031,0.537548,0.573121,0.571802
l_clean_phone,-0.106469,-0.033208,-0.047452,-0.181321,1.000000,-0.000734,-0.008646,-0.034268,0.027788,0.019352,0.052605,0.020639,0.333074,0.016184,0.096345,0.022053,0.148969,0.082904
r_clean_phone,-0.002840,0.007791,-0.005460,-0.011031,-0.000734,1.000000,-0.006856,-0.009696,-0.007908,-0.010923,-0.011003,0.006297,0.022713,-0.003534,-0.003604,-0.010676,-0.001024,-0.003019
jaro_score,0.241467,-0.152362,0.394864,0.405832,-0.008646,-0.006856,1.000000,0.886886,0.722620,0.761749,0.711259,-0.251768,-0.199746,0.452747,0.214556,0.862252,0.518336,0.451593
fuzz_partial_score,0.196616,-0.074333,0.373749,0.434351,-0.034268,-0.009696,0.886886,1.000000,0.719862,0.758955,0.736629,-0.236670,-0.138848,0.535758,0.261033,0.875616,0.590654,0.537366
fuzz_sort_score,0.183734,0.052334,0.244038,0.504531,0.027788,-0.007908,0.722620,0.719862,1.000000,0.927519,0.807405,-0.157341,-0.047542,0.588630,0.311418,0.907374,0.732655,0.641346
fuzz_set_score,0.241088,-0.098783,0.264388,0.457027,0.019352,-0.010923,0.761749,0.758955,0.927519,1.000000,0.838308,-0.244183,-0.132321,0.556927,0.268325,0.934846,0.672950,0.580203


In [5]:
feature_cols = ['jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                #'domain_match',
                'phone_match']

# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(jaro_reduced[feature_cols])
y_pred_proba = log.predict_proba(jaro_reduced[feature_cols])[:,1]

jaro_reduced['match_pred'] = y_pred_class
jaro_reduced['pred_proba'] = y_pred_proba

#we take any matches meeting either name match threshold or composite match threshold as matches for review
#org_matches = jaro_reduced[(jaro_reduced.overall_name_score >= name_score_min) | (jaro_reduced.composite_match_score >= composite_score_min)]
org_matches = jaro_reduced[jaro_reduced.match_pred == 1]
failed_matches_for_review = jaro_reduced[(jaro_reduced.match_pred == 0) & (jaro_reduced.pred_proba > .4)]

print("final matches isolated --- %s seconds ---" % (time.time() - start_time))
print ""

print("TOTAL COMPUTE TIME --- %s seconds ---" % (time.time() - overall_time))

org_matches.sort_values(by='pred_proba', ascending=False)

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


final matches isolated --- 1964.28399992 seconds ---

TOTAL COMPUTE TIME --- 6533.59200001 seconds ---


,id_l,id_r,id_l count,l_org_name,l_city,l_state,l_postal_code,l_domain,l_clean_phone,r_org_name,...,fuzz_set_score,uniq,state_match,zip_match,domain_match,phone_match,overall_name_score,composite_match_score,match_pred,pred_proba
4939769,28870,24140,10,Northern Natural Gas Company,Omaha,ne,68124,northernnaturalgas.com,4023987200,Northern Natural Gas Company ...,...,1.00,1.000000,1,1,1.0,1,4.378000,10.378000,1,0.994160
5493588,91786,19848,11,Peninsular Florida District Council/Assemblies...,Lakeland,fl,33801,penflorida.org,8636835726,Peninsular Florida District Council/Assemblie...,...,1.00,1.000000,1,1,1.0,1,4.382163,10.382163,1,0.994123
4629856,95304,13968,10,Wisconsin Evangelical Lutheran Synod,Waukesha,wi,53188,els.net,4142563888,Wisconsin Evangelical Lutheran Synod,...,1.00,1.000000,1,1,1.0,1,4.382675,10.382675,1,0.994119
4713249,88362,18238,9,International Association of Information Techn...,Canton,oh,44718,iaitam.org,3306283012,International Association of Information Tech...,...,1.00,1.000000,1,1,1.0,1,4.386041,10.386041,1,0.994089
5491433,88496,19449,15,International Myopain Society,Nine Mile Falls,wa,99026,myopain.org,7144234863,International Myopain Society,...,1.00,1.000000,1,1,1.0,1,4.396839,10.396839,1,0.993994
5272450,87546,17852,8,Freedom Socialist Party,Seattle,wa,98105,socialism.com,2069854621,Freedom Socialist Party,...,1.00,1.000000,1,1,1.0,1,4.402627,10.402627,1,0.993942
5505953,60116,42251,12,California Society of Accounting and Tax Profe...,Sacramento,ca,95819-4608,csatp.org,8008948995,California Society of Accounting and Tax Prof...,...,1.00,1.000000,1,1,1.0,1,4.402862,10.402862,1,0.993940
3382798,90138,19482,12,Multiservice Management Company Pennsylvania,Wayne,pa,19087-1866,mmco1.com,6109714850,Multiservice Management Company Pennsylvania,...,1.00,1.000000,1,1,1.0,1,4.403535,10.403535,1,0.993934
4700512,56385,40204,10,South Dakota Nurses Association,Pierre,sd,57501-1015,sdnursesassociation.org,6059454265,South Dakota Nurses Association,...,1.00,1.000000,1,1,1.0,1,4.411542,10.411542,1,0.993861
4579602,18454,47836,11,Casey Family Programs,Seattle,wa,98121,"casey.org, www.casey.org",2069927254,Casey Family Programs,...,1.00,1.000000,1,1,NaN,1,4.417208,NaN,1,0.993809


In [3]:
jaro_reduced.isnull().sum()

id_l                           0
id_r                           0
id_l count                     0
l_org_name                     0
l_city                         0
l_state                        0
l_postal_code                  0
l_domain                       0
l_clean_phone                  0
r_org_name                     0
r_city                         0
r_state                        0
r_postal_code                  0
r_domain                       0
r_clean_phone                  0
jaro_score                     0
fuzz_partial_score             0
fuzz_sort_score                0
fuzz_set_score                 0
uniq                           0
state_match                    0
zip_match                      0
domain_match             5429507
phone_match                    0
overall_name_score             0
composite_match_score    5429507
dtype: int64